Exemplo retirado de: https://github.com/wradlib/wradlib-notebooks/blob/main/notebooks/beamblockage/beamblockage.ipynb

# Preparação do ambiente

In [ ]:
# instala bibliotecas
!pip install -q wradlib wradlib-data

# importa bibliotecas
import wradlib as wrl
import wradlib_data
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# monta drive
from google.colab import drive
drive.mount('/content/drive')

# caminho do drive
dir = '/content/drive/MyDrive/6-COMPRA_RADAR/02_CODIGOS'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.1/235.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.6 MB/s eta 0:00:00


# Definindo funções

In [ ]:
# FUNÇÃO: just a little helper function to style x and y axes of our maps
def annotate_map(ax, cm=None, title=""):

    ticks = (ax.get_xticks() / 1000).astype(int)

    ax.set_xticklabels(ticks)

    ticks = (ax.get_yticks() / 1000).astype(int)

    ax.set_yticklabels(ticks)
    ax.set_xlabel("Kilometers")
    ax.set_ylabel("Kilometers")

    #if not cm is None:
    #    plt.colorbar(cm, ax=ax)

    if not title == "":
        ax.set_title(title)

    ax.grid()

# FUNÇÃO: calcula o azimute entre LNA e Itajubá
def get_distance_and_azimuth(lon1, lat1, lon2, lat2):

    """
    Função que calcula a distância (km) e azimute (graus) entre dois pontos

    Parâmetros:
                lat1 (float): latitude em graus do ponto 1
                lon1 (float): longitude em graus do ponto 1
                lat2 (float): latitude em graus do ponto 2
                lon2 (float): longitude em graus do ponto 2

    Retorna:
             fwd_azimuth (float): ângulo azimute "para frente" em graus
             back_azimuth (float): ângulo azimute "para trás" em graus
             distancia_km (float): distância em km entre o ponto 1 (lat1, lon1) e ponto 2 (lat2, lon2)
    """

    import pyproj

    geodesic = pyproj.Geod(ellps='WGS84')

    fwd_azimuth_graus, back_azimuth_graus, distance_m = geodesic.inv(lon1, lat1, lon2, lat2)

    distance_km = distance_m/1000.

    return fwd_azimuth_graus, back_azimuth_graus, distance_km
#fwd_azimuth_graus, back_azimuth_graus, distance_km = get_distance_and_azimuth(lon_radar, lat_radar, lon_itajuba, lat_itajuba)
#print(fwd_azimuth_graus, back_azimuth_graus, distance_km)

# Processamento e figura

In [ ]:
%%time
#==============================================================================================================#
#                                          INFORMAÇÕES DO RADAR
#==============================================================================================================#
# coordenadas de itajubá
lat_itajuba, lon_itajuba = -22.42, -45.45

# coordenadas do radar (que fica no LNA, cidade de Brazópolis-MG)
lat_radar, lon_radar, alt_radar = -22.5344, -45.5825, 1850
sitecoords = (lon_radar, lat_radar, alt_radar)  # (longitude, latitude, altitude)

# azimutes, bins, abertura do feixe e resolução radial
nrays = 360                # number of rays
nbins = 960                # number of range bins
radar_beam_width = 0.968   # largura do feixe [graus]
bw =  radar_beam_width/2.  # half power beam width (deg) obs. atentar que é metade
range_res = 250            # range resolution [metros]

# define as elevaçoes [graus]. Exemplo: [-0.5, -0.4, -0.3, -0.2, -0.1, -0. ,  0.1,  0.2,  0.3,  0.4,  0.5, 0.6,  0.7,  0.8,  0.9,  1.0]
elevacoes = np.round(np.arange(-0.5, 1.1, 0.1), 1)
elevacoes[5]=0.0

#==============================================================================================================#
#                               REALIZA O PROCESSAMENTO E GERA A FIGURA
#==============================================================================================================#
# loop nas elevações
for t, elevacao in enumerate(elevacoes):

    #----------------------------------------------------------------#
    #               CALCULANDO AS COORDENADAS DOS BINS
    #----------------------------------------------------------------#
    # cria o array do range e raio do beam
    r = np.arange(nbins) * range_res               # distância de cada bin ao radar. Para o Banda-S 0-239750 [metros]
    beamradius = wrl.util.half_power_radius(r, bw) # retorna o Half-power radius [metros]

    # calcula as coordenadas nativas do centro do bin para os bins de uma determinada "ELEVAÇÃO"
    coord = wrl.georef.sweep_centroids(nrays,
                                       range_res,
                                       nbins,
                                       elevacao)

    # transforma coordenadas esféricas (r, phi, theta) em coordenadas projectadas centradas no local numa dada projeção
    coords = wrl.georef.spherical_to_proj(coord[..., 0],
                                          coord[..., 1],
                                          coord[..., 2],
                                          sitecoords)

    # longitude, latitude a altitude de cada bin daquela elevação
    lon = coords[..., 0] # longitude do bin [graus]
    lat = coords[..., 1] # latitude do bin [graus]
    alt = coords[..., 2] # altitude do bin [metros]

    # limites dos dados. Exemplo: [-47.9117, -24.6977, -43.2532, -20.37048]
    rlimits = (lon.min(), lat.min(), lon.max(), lat.max())

    #----------------------------------------------------------------#
    #                PROCESSANDO OS DADOS DE TOPOGRAFIA
    #----------------------------------------------------------------#
    # O mapa de topografia para a região do radar deve ser baixado em formato TIFF do site: "https://download.gebco.net/"
    # Para baixar o dado, realize o seguinte passo-a-passo:
    #   1. Acessar o site do dado de relevo: https://download.gebco.net/
    #   2. Em "Enter boundaries" inserir os limites da região de interesse
    #   3. Em "Enter boundaries" escolher o formato GeoTIFF - Opção Grid
    #   4. Clicar em "Add to basket", seguido por "View basket" e em "Download your data"
    #   5. Descompctar a pasta baixada e inserir o arquivo TIFF na pasta de entrada. Exemplo: gebco_2024_n-20.3705_s-24.6977_w-47.9117_e-43.2532.tif
    # Uma segunda opção de site: "https://earthexplorer.usgs.gov/" data for GTOPO30

    # salva a altitude dos bins na variável "polcoords"
    polcoords = coords[..., :2]

    # nome do arquivo da topografia que foi baixado
    rasterfile = f"{dir}/gebco_2021_n-15.0_s-30.0_w-55.0_e-40.0.tif"

    # abre o arquivo de topografia
    ds = wrl.io.open_raster(rasterfile)

    # extrae as coordenadas do arquivo de topografia
    rastervalues, rastercoords, crs = wrl.georef.extract_raster_dataset(ds, nodata=-32768.0) # altitudes, [lons, lats] e projeção

    # pega os índices das coordenadas que pertencem a região
    ind = wrl.util.find_bbox_indices(rastercoords, rlimits)

    # seleciona as coordenadas e altitudes daqueles índices pertencem a região
    rastercoords = rastercoords[ind[1] : ind[3], ind[0] : ind[2], ...] # coordenadas de longitude e latitude
    rastervalues = rastervalues[ind[1] : ind[3], ind[0] : ind[2]]      # altitudes

    # mapea os valores raster para pontos em grade polar
    polarvalues = wrl.ipol.cart_to_irregular_spline(rastercoords,
                                                    rastervalues,
                                                    polcoords,
                                                    order=3,
                                                    prefilter=False)

    #----------------------------------------------------------------#
    #          CALCULANDO O BLOQUEIO DO FEIXE (BEAMBLOCKAGE)
    #----------------------------------------------------------------#
    # Fração do beam que sofre bloqueio do feixe
    PBB = wrl.qual.beam_block_frac(polarvalues,
                                   alt,
                                   beamradius)
    PBB = np.ma.masked_invalid(PBB)

    # Fração acumulada do beam que sofre bloqueio do feixe
    # Na linha anterior calculamos a fração do beam blockage de cada bin. Mas temos de ter em conta que o sinal de radar viaja ao longo de um feixe.
    # O bloqueio cumulativo do feixe (CBB) num bin ao longo de um feixe será sempre, pelo menos,
    # tão elevado como o PBB máximo dos bins anteriores (ver - wradlib.qual.cum_beam_block_frac: https://docs.wradlib.org/en/latest/generated/wradlib.qual.cum_beam_block_frac.html)
    CBB = wrl.qual.cum_beam_block_frac(PBB)

    # mostra o formato na tela
    #print(PBB.shape)
    #print(CBB.shape)

    #----------------------------------------------------------------#
    #                  MAPA DO BEAM BLOCKAGE
    #----------------------------------------------------------------#
    # moldura da figura
    fig = plt.figure(figsize=(15, 12))

    # cria subplots
    ax2 = plt.subplot2grid((2, 2), (0, 1))

    # plota CBB (no ax2)
    CBB = wrl.georef.create_xarray_dataarray(CBB, r=r, phi=coord[:, 0, 1]).wrl.georef.georeference()
    cbb = CBB.wrl.vis.plot(ax=ax2, cmap=mpl.cm.PuRd, vmin=0, vmax=1)
    annotate_map(ax2, cbb, f"Beam-Blockage Fraction / Elevation = {elevacao}$\degree$")

    # recorta figura
    plt.tight_layout()

    # salva figura
    plt.savefig(f'{dir}/output/codigo_4/{str(t+1).zfill(2)}_beamblock_BandaS_elevacao_{elevacao}graus.jpg', dpi=300, bbox_inches='tight')